# Experiment Dashboard

Interactive dashboard for exploring experiments. Works as both a Jupyter notebook and a Voila web app.


In [ ]:
import os
from amprenta_rag.client import RAGClient
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Connect to API
api_url = os.environ.get('API_URL', 'http://host.docker.internal:8000')
client = RAGClient(api_url=api_url)
print(f'Connected to {api_url}')


In [ ]:
# Load experiments
experiments = client.experiments.list()
print(f'Loaded {len(experiments)} experiments')

# Create dropdown options
if experiments:
    experiment_options = [(f"{exp.name} ({str(exp.id)[:8]}...)", exp.id) for exp in experiments]
else:
    experiment_options = [("No experiments found", None)]


In [ ]:
# Create widgets
experiment_dropdown = widgets.Dropdown(
    options=experiment_options if experiments else [("No experiments", None)],
    description='Experiment:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

output_area = widgets.Output()

def update_dashboard(experiment_id):
    """Update dashboard when experiment is selected."""
    with output_area:
        clear_output(wait=True)
        
        if not experiment_id:
            print("Please select an experiment.")
            return
        
        try:
            # Fetch experiment details
            exp = client.experiments.get(experiment_id)
            
            # Display experiment details
            print(f"## {exp.name}")
            print(f"\n**ID:** {exp.id}")
            if hasattr(exp, 'description') and exp.description:
                print(f"**Description:** {exp.description}")
            if hasattr(exp, 'omics_type') and exp.omics_type:
                print(f"**Omics Type:** {exp.omics_type}")
            if hasattr(exp, 'created_at') and exp.created_at:
                print(f"**Created:** {exp.created_at}")
            
            # Create sample visualization
            fig, ax = plt.subplots(figsize=(8, 4))
            
            # Mock data for demonstration
            # In real usage, fetch actual experiment data
            x = np.linspace(0, 10, 100)
            y = np.sin(x) + np.random.normal(0, 0.1, 100)
            
            ax.plot(x, y, alpha=0.7, label='Sample Data')
            ax.set_xlabel('Time')
            ax.set_ylabel('Value')
            ax.set_title(f'Experiment: {exp.name[:50]}')
            ax.legend()
            ax.grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.show()
            
            print("\n*Note: Replace mock visualization with actual experiment data.*")
            
        except Exception as e:
            print(f"Error loading experiment: {e}")

# Link dropdown to update function
experiment_dropdown.observe(
    lambda change: update_dashboard(change['new']),
    names='value'
)

# Display widgets
display(experiment_dropdown)
display(output_area)

# Initial update if experiment is selected
if experiment_dropdown.value:
    update_dashboard(experiment_dropdown.value)


## Usage

1. Select an experiment from the dropdown above
2. The dashboard will automatically update with experiment details and visualizations
3. To run as a Voila app: `voila experiment_dashboard.ipynb`
